# Threads

In [ ]:
import _thread
import threading
import time

### 1. What is a thread?
A thread is a sequence of instructions executed one by one. 

In the example below you see a test_function. This function counts to 5 and shows the time from the start of the run. The delay shows how long it has to wait between prints. 

Two test_functions are executed one after another, one with delay 1s, one with delay 2s. In total this will thus take 5x1 + 5x2 = 15s to run. The functions are executed in sequence, and are thus all executed on one thread. 

In [ ]:
def test_function(name, delay, starttime):
    for count in range(5):
        time.sleep(delay)
        print("%s: %s" % (name, time.time()-starttime))

start = time.time()
test_function('Test 1', 1, start)
test_function('Test 2', 2, start)


### Implementation of thread
The code below shows the implementation of threading using the thread module (this is a Python 2 module, recommended now is the threading module). The function 'start_new_thread' starts a new thread and takes as input the function that should be executed on that thread, as well as the inputs that should be given to that function. 
Look at the output. How long does it take to execute both functions? Is this what you expected?

In [ ]:
start = time.time()
_thread.start_new_thread(test_function,('Test 1', 1, start))
_thread.start_new_thread(test_function,('Test 2', 2, start))

### Thread with caution
Threads are able to share resources. When two threads are modifying the same variable concurrently, you might get results you didn't expect. To show this, we're going to define two new functions. The first takes a global variable (i) and adds one to it on each iteration. The seconds takes a global variable (i) and multiplies it by 2 on each iteration. 
We're going to run these function in sequence. What will the result be? Is this always the same results?

In [ ]:
global i
i = 0

def add_function(name):
    for count in range(5):
        global i
        i = i+1
        time.sleep(1)
        print("%s: %s" % (name, i))

def multiple_function(name):
    for count in range(5):
        global i
        i = i*2
        time.sleep(1)
        print("%s: %s" % (name, i))


add_function('Add')
multiple_function('Multiply')

The final value of i in the above code will always be 160. However, when we try to use threading to run these two functions concurrently, the answer will be different. Try running the code below a few times and see what the result is. Is the result the same for each run? 

In [ ]:
i = 0
_thread.start_new_thread(add_function,('Add',))
_thread.start_new_thread(multiple_function,('Multiply',))

To combat the above problem, you can use the join function. This causes the main thread to wait until the other thread is done executing. You can also use a lock. 

### Join function
If you want your code to wait to finish executing one thread before executing the next, you can use the 'join' function. This is not available in the thread module, so the next example will use the threading module. To use this, we have to write a Class. If you are not familiar with OO programming, don't worry about the exact implementation of the threads.

The implementation below does the same as the code above. It starts two threads, one running the add function, the other running the multiply function. If you run it multiple times, you will notice this implementation is slightly more reliable (which is why it is advised), but will still not produce the same answer as the serial implementation.

In [ ]:
class myThread (threading.Thread):
    def __init__(self, name, func):
        threading.Thread.__init__(self)
        self.name = name
        self.func = func
    def run(self):
        print("Starting " + self.name)
         # Get lock to synchronize threads
        if self.func == 1:
            add_function(self.name)
        else:
            multiple_function(self.name)
            
global i
i = 0

# Create threads        
thread1 = myThread('Add',1)
thread2 = myThread('Multiply',2)

# Start new Threads
thread1.start()
thread2.start()


We are now going to use the join function. This makes sure thread 2 doesn't run until thread 1 has finished running. The description of the join function is:

*Wait until the thread terminates. This blocks the calling thread until the thread whose join() method is called terminates – either normally or through an unhandled exception – or until the optional timeout occurs.*

Run the code below. What answer do you get?

In [ ]:
global i
i = 0

# Create threads        
thread1 = myThread('Add',1)
thread2 = myThread('Multiply',2)

# Start new Threads
thread1.start()
thread1.join()
thread2.start()

For our last example we're going to change to change the code slighly to have an extra function. This function is called infinity_add, and will be an infinite loop. You will notice that if we use the join function, thread 2 will never start running, because thread 1 is in an infinite loop. 

In [ ]:
def infinity_add(name):
    while True:
        global i
        i = i+1
        time.sleep(1)
        print("%s: %s" % (name, i))

class myInfThread (threading.Thread):
    def __init__(self, name, func):
        threading.Thread.__init__(self)
        self.name = name
        self.func = func
    def run(self):
        print("Starting " + self.name)
         # Get lock to synchronize threads
        if self.func == 1:
            infinity_add(self.name)
        else:
            print("I give some vital information")
            

In [ ]:
## Don't run this, will get stuck in infinite loop!
global i
i = 0

# Create threads        
thread1 = myInfThread('Add',1)
thread2 = myInfThread('Multiply',2)

# Start new Threads
thread1.start()
thread1.join()
thread2.start()

Though the infinite loop was fabricated in this case, you can imagine it might happen on accident in a GUI, or when asking users for input. For this we can use a timeout. We basically will force the thread to join after a certain amount of time. You can just input the time in the join function as shown below. Note that the first thread will still keep running, but the second thread now did run as well, giving you the vital information you would have otherwise not gotten

In [ ]:
global i
i = 0

# Create threads        
thread1 = myInfThread('Add',1)
thread2 = myInfThread('Multiply',2)

# Start new Threads
thread1.start()
thread1.join(5)
thread2.start()
